In [4]:
# khai báo thư viện
import pandas as pd
from scipy import  stats
# đọc bộ dữ liệu
df = pd.read_csv("subset-covid-data.csv")
df.describe()

,day,month,year,cases,deaths,population
count,206.000000,206.000000,206.0,206.000000,206.000000,2.020000e+02
mean,12.024272,3.985437,2020.0,395.247573,29.713592,3.731716e+07
std,0.424141,0.209020,0.0,2160.176439,159.533131,1.413776e+08
min,11.000000,1.000000,2020.0,0.000000,0.000000,1.000000e+03
25%,12.000000,4.000000,2020.0,0.000000,0.000000,1.278222e+06
50%,12.000000,4.000000,2020.0,11.500000,0.000000,7.042862e+06
75%,12.000000,4.000000,2020.0,88.750000,3.000000,2.600084e+07
max,18.000000,4.000000,2020.0,28391.000000,1831.000000,1.392730e+09


#### Có mối tương quan nào giữa số ca mắc và ca tử vong ở các quốc gia hay không
Do số ca nhiễm (cases) và dân số (population) đều là thuộc tính định lượng nên sử dụng kiểm định pearson
Giả thuyết không: Không có mối tương quan tuyến tính giữa hai biến
Giả thuyết đối: Không có mối tương quan tuyến tính giữa hai biến

In [5]:
# lọc dữ liệu
df1 = df.filter(['cases', 'population'])
# xóa bỏ dữ liệu null
df1 = df1.dropna()
r, pvalue = stats.pearsonr(df1.cases, df1.population)
print ("r: ", r, "; pvalue: ", pvalue)

r:  0.17553926594294003 ; pvalue:  0.01246031070420893


##### Nhận xét: do pvalue <5%, nên với mức ý nghĩa 5% bác bỏ giả thuyết không, chấp nhận giả thuyết đối
##### Kết luận: Giữa dân số và số ca nhiễm có tương quan tuyến tính yếu với nhau

#### Có mối tương quan nào giữa số ca mắc (cases) và số ca tử vong (deaths) hay không
Do số ca nhiễm (cases) và số ca tử vong (deaths) đều là thuộc tính định lượng nên sử dụng kiểm định pearson
Giả thuyết không: Không có mối tương quan tuyến tính giữa hai biến
Giả thuyết đối: Không có mối tương quan tuyến tính giữa hai biến

In [6]:
# lọc dữ liệu
df2 = df.filter(['cases', 'deaths'])
# xóa bỏ dữ liệu null
df2 = df2.dropna()
r, pvalue = stats.pearsonr(df2.cases, df2.deaths)
print ("r: ", r, "; pvalue: ", pvalue)

r:  0.9432201374746727 ; pvalue:  1.3451948745846463e-99


##### Nhận xét: do pvalue ~0, nên với mức ý nghĩa 5% bác bỏ giả thuyết không, chấp nhận giả thuyết đối
##### Kết luận: Giữa số ca mắc và số ca tử vong có tương quan tuyến tính mạnh với nhau

#### Biến đổi lại thuộc tính population về dạng thứ bậc thực hiện kiểm định tương quan giữa biến này với số ca nhiễm

In [7]:
# tiến hành tính các khoảng tứ phân vị của population
q1, q2, q3  = df1.population.quantile(0.25), df1.population.quantile(0.5), df1.population.quantile(0.75)
# tiến hành biến đổi population
def population_order(population):
    if population < q1:
        return 1
    elif population>=q1 and population <q2:
        return 2
    elif population>=q2 and population <q3:
        return 3
    else: 
        return 4

df1['population_ordinal']=df1.population.apply(population_order)
df1.head()

,cases,population,population_ordinal
0,34,37172386.0,4
1,17,2866376.0,2
2,64,42228429.0,4
3,21,77006.0,1
4,0,30809762.0,4


#### Tiến hành kiểm định. Do population_ordinal là dữ liệu kiểu thứ bậc nên sử dụng kiểm định spearman để kiểm định tương quan giữa population_ordinal và cases
Giả thuyết không: Giữa population_ordinal và case không có tương quan Giả thuyết đối: Giữa population_ordinal và case có tương quan

In [8]:
r, pvalue = stats.spearmanr(df1.cases, df1.population_ordinal)
print ("r: ", r, "; pvalue: ", pvalue)

r:  0.49498466493711596 ; pvalue:  6.982786622314858e-14


#### Pvalue ~0, và r~0.5 chứng tỏ giữa thuộc tính population_ordinal và số ca nhiễm bệnh có tương quan với nhau. Như vậy có thể thấy, việc biến đổi dữ liệu thành các khoảng có thể làm nổi bật thông tin tương quan giữa cá thuộc tính.